In [1]:
import os
import tensorflow as tf
import dask
import dask.array as da
import numpy as np
from PIL import Image
from skimage.transform import resize
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
train_x = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/train_x')
train_y = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/train_y')
test_x = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/test_x')
test_y = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/test_y')
test_fov = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/test_fov')
train_x, train_y, test_x, test_y, test_fov

(dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>,
 dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>,
 dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>,
 dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>,
 dask.array<from-npy-stack, shape=(20, 512, 512), dtype=float32, chunksize=(20, 512, 512)>)

In [3]:
def build_block(input_layer, filters, norm=True, k=(3, 3)):
    layer = tf.keras.layers.Conv2D(filters, kernel_size=k, padding='same', use_bias=not norm, kernel_initializer='glorot_normal')(input_layer)
    if norm:
        layer = tf.keras.layers.BatchNormalization()(layer)
    layer = tf.keras.layers.Activation('elu')(layer)
    return layer

def build_unet(input_shape, n_filters=16, dropout=0.2):
    image_input = tf.keras.Input(shape=input_shape, name='input_layer')
    
    conv_1 = build_block(image_input, n_filters)
    conv_2 = build_block(conv_1, n_filters)
    pool_1 = tf.keras.layers.AveragePooling2D(padding='same')(conv_2)
    drop_1 = tf.keras.layers.SpatialDropout2D(dropout)(pool_1)
    
    conv_3 = build_block(drop_1, n_filters * 2)
    conv_4 = build_block(conv_3, n_filters * 2)
    pool_2 = tf.keras.layers.AveragePooling2D(padding='same')(conv_4)
    drop_2 = tf.keras.layers.SpatialDropout2D(dropout)(pool_2)
    
    conv_5 = build_block(drop_2, n_filters * 4)
    conv_6 = build_block(conv_5, n_filters * 4)
    pool_3 = tf.keras.layers.AveragePooling2D(padding='same')(conv_6)
    drop_3 = tf.keras.layers.SpatialDropout2D(dropout)(pool_3)
    
    conv_7 = build_block(drop_3, n_filters * 8)
    conv_8 = build_block(conv_7, n_filters * 8)
    pool_4 = tf.keras.layers.AveragePooling2D(padding='same')(conv_8)
    drop_4 = tf.keras.layers.SpatialDropout2D(dropout)(pool_4)
    
    conv_9 = build_block(drop_4, n_filters * 16)
    conv_10 = build_block(conv_9, n_filters * 16)
    
    upsp_1 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_10) #(-1, 8, 8, 256)
    upsp_1 = tf.keras.layers.concatenate([upsp_1, conv_8])#(-1, 8, 8, 384)
    conv_11 = build_block(upsp_1, n_filters * 8)#(-1, 8, 8, 128)
    conv_12 = build_block(conv_11, n_filters * 8)#(-1, 8, 8, 128)
    drop_5 = tf.keras.layers.SpatialDropout2D(dropout)(conv_12)#(-1, 8, 8, 128)
    
    upsp_2 = tf.keras.layers.UpSampling2D(size=(2, 2))(drop_5) #(-1, 16, 16, 128)
    upsp_2 = tf.keras.layers.concatenate([upsp_2, conv_6]) #(-1, 16, 16, 192)
    conv_13 = build_block(upsp_2, n_filters * 4) #(-1, 16, 16, 64)
    conv_14 = build_block(conv_13, n_filters * 4) #(-1, 16, 16, 64)
    drop_6 = tf.keras.layers.SpatialDropout2D(dropout)(conv_14) #(-1, 16, 16, 64)
    
    upsp_3 = tf.keras.layers.UpSampling2D(size=(2, 2))(drop_6) #(-1, 32, 32, 64)
    upsp_3 = tf.keras.layers.concatenate([upsp_3, conv_4]) #(-1, 32, 32, 96)
    conv_15 = build_block(upsp_3, n_filters * 2) #(-1, 32, 32, 32)
    conv_16 = build_block(conv_15, n_filters * 2) #(-1, 32, 32, 32)
    drop_7 = tf.keras.layers.SpatialDropout2D(dropout)(conv_16) #(-1, 32, 32, 32)
    
    upsp_4 = tf.keras.layers.UpSampling2D(size=(2, 2))(drop_7) #(-1, 64, 64, 32)
    upsp_4 = tf.keras.layers.concatenate([upsp_4, conv_2])#(-1, 64, 64, 48)
    conv_17 = build_block(upsp_4, n_filters * 2)#(-1, 64, 64, 16)
    conv_18 = build_block(conv_17, n_filters * 2)#(-1, 64, 64, 16)
    drop_8 = tf.keras.layers.SpatialDropout2D(dropout)(conv_18)#(-1, 64, 64, 16)
    
    output = tf.keras.layers.Conv2D(1, (1, 1), kernel_initializer='glorot_normal', activation='sigmoid')(drop_8)
    model = tf.keras.Model(inputs=image_input, outputs=output)
    return model

def random_crop(img, msk, random_crop_size):
    # Note: image_data_format is 'channel_last'
    #assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :], msk[y:(y+dy), x:(x+dx), :]


def crop_generator(batches, crop_length):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    while True:
        batch_x, batch_y = next(batches)
        batch_x_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, batch_x.shape[3]))
        batch_y_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, batch_x.shape[3]))
        for i in range(batch_x.shape[0]):
            batch_x_crops[i], batch_y_crops[i] = random_crop(batch_x[i], batch_y[i], (crop_length, crop_length))
        yield (batch_x_crops, batch_y_crops)
        
model = build_unet(input_shape=(None, None, 1))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, None, None, 1 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 1 144         input_layer[0][0]                
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 1 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 1 0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d_1 (

In [6]:
model.load_weights("/home/skyolia/JupyterProjects/segmentation/DRIVE/base_2/day_2.weights.best.hdf5")
opt = tf.keras.optimizers.Adam() # 
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
scores = model.evaluate(test_x, test_y, batch_size=1)
scores

20/20 [==============================] - 2s 76ms/step


[0.0853958860039711, 0.9226659774780274]

In [24]:
pred = model.predict(x=test_x,batch_size=1)
pred.shape

(20, 512, 512, 1)

In [9]:
test_fov = da.where(test_fov>0.5, 1, 0)
da.unique(test_fov).compute()

array([0, 1])

In [19]:
len(test_fov.ravel().compute())

5242880

In [20]:
def get_inside_fov(test_fov):
    flat = test_fov.ravel()
    flat = da.where(flat>0.5, 1, 0)
    index_inside_fov = da.where(flat)[0]
    return index_inside_fov
inside_fov = get_inside_fov(test_fov).compute()
inside_fov

array([  14069,   14070,   14071, ..., 5235971, 5235972, 5235973])

In [34]:
from sklearn.metrics import roc_curve, auc, roc_auc_score, f1_score, confusion_matrix
bin_test_y = np.where(test_y.compute()>0.5, 1, 0)
fpr_keras, tpr_keras, thresholds_keras = roc_curve(bin_test_y.ravel()[inside_fov], pred.ravel()[inside_fov])
auc_keras = auc(fpr_keras, tpr_keras)
auc_roc = roc_auc_score(bin_test_y.ravel()[inside_fov], pred.ravel()[inside_fov])
auc_keras, auc_roc

(0.9786056932279753, 0.9786056932279753)

In [35]:
from sklearn.metrics import classification_report
classification_report(bin_test_y.ravel()[inside_fov], pred.ravel()[inside_fov])

ValueError: Classification metrics can't handle a mix of binary and continuous targets